In [52]:
import torch
import numpy as np
import pandas as pd
import transformers
# import sys
# import os
# src_path = os.path.abspath(os.path.join(os.getcwd(), '..', ''))
# sys.path.append(src_path)
from transformers import BertTokenizer, AutoTokenizer
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [53]:
import torch
from torch.utils.data import Dataset
import re

class AutomaticScoringDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def preprocess_text(self, text):
        text = ' '.join(text.split())  # Hapus spasi berlebih
        text = text.lower()  # Ubah ke lowercase
        text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)  # Hapus karakter khusus
        return text

    def __getitem__(self, index):
        student_answer = self.preprocess_text(str(self.dataframe.iloc[index]['answer']))
        score = self.dataframe.iloc[index]['normalized_score']
        reference_answer = self.preprocess_text(str(self.dataframe.iloc[index]['reference_answer']))
        encoding = self.tokenizer.encode_plus(
            reference_answer,
            student_answer,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        encoding = {key: tensor.squeeze(0) for key, tensor in encoding.items()}
        encoding['labels'] = torch.tensor(score, dtype=torch.float)

        return encoding

In [54]:
import torch
import torch.nn as nn
from transformers import AlbertConfig
from modeling_albert_default import AlbertModel

class RegressionModel(nn.Module):
    def __init__(self, model_name='bert-base-uncased', classifier_dropout=0.1, hidden_dropout=0.0, attention_dropout=0.0):
        super().__init__()
        # Load model config and modify dropout values
        self.config = AlbertConfig.from_pretrained(model_name)
        self.config.hidden_dropout_prob = hidden_dropout
        self.config.attention_probs_dropout_prob = attention_dropout
        
        # Load pretrained model
        self.model = AlbertModel.from_pretrained(model_name, config=self.config)
        
        # Add regression layer
        self.dropout = nn.Dropout(p=classifier_dropout)
        self.regression_layer = nn.Linear(self.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.model(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
        
       
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls_embedding)
        score = self.regression_layer(x)
        return score

In [ ]:
class inference:
    def __init__(self, df, model_name, model_dir):
        self.model = RegressionModel(model_name).to(device)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        if(model_dir != ""):
            self.model.load_state_dict(torch.load(model_dir, weights_only=True))
        self.dataset = AutomaticScoringDataset(df, self.tokenizer)

    def get_prediction(self, index):
        self.model.eval()
        data = self.dataset[index]
        label = round(data['labels'].item(),2)
        data_cuda = {k: v.to(device) for k, v in data.items()}
        input_ids = data_cuda['input_ids'].unsqueeze(0)
        attention_mask = data_cuda['attention_mask'].unsqueeze(0)
        token_type_ids = data_cuda['token_type_ids'].unsqueeze(0)
        with torch.no_grad():
            predictions = self.model(input_ids, attention_mask, token_type_ids)

        predicted_score = round(predictions.squeeze().item(), 2)
        return label, predicted_score

In [46]:
data = {
    "reference_answer": [
        "Fungsi karbohidrat adalah sebagai pemasok energi, dapat memperlancar proses pada pencernaan, memberikan efek kenyang dengan kandungan selulosa-nya dan penyeimbang asam dan basa dalam tubuh"
    ] * 5,
    "answer": [
        # "Untuk sumber energi, Untuk pembentuk sel, Sebagai sumber tenaga, Sebagai pembentuk daging",
        # "sumber tenaga, pemanis alami, menjaga sistem imun, dan sebagai keseimbangan tubuh",
        # "Karbohidrat mempunyai peran penting untuk proses metabolisme atau proses yang berfungsi sebagai penyeimbang asam dan basa dalam tubuh. Karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak. Fungsi karbohidrat yang utama adalah sebagai pemasok energi. Makanan berkarbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan.",
        # "mendapat tenaga",
        # "Mempunyai peran penting untuk proses metabolisme, atau proses yang berfungsi sebagai penyeimbang asam dan basa di dalam tubuh, serta proses untuk membentuk jaringan sel, struktur dan juga organ-organ dalam tubuh. Karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak Fungsi karbohidrat yang utama adalah sebagai pemasok energi. Dalam 1 gram karbohidrat mampu menghasilkan 4 kkalori yang melalui proses pembakaran kimiawi di dalam tubuh akan menjadi sumber energi. ,Makanan ber-karbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan Karena sifat kandungan yang ada dalam karbohidrat yang manis, maka fungsi karbohidrat kali ini adalah sebagai pemanis alami, khususnya dengan kandungan monosakarida dan disakarida Fungsi karbohidrat juga berperan penting untuk mengoptimalkan kerja protein. Karena apabila tubuh mengalami kekurangan asupan karbohidrat, maka protein akan menggantikan fungsi karbohidrat sebagai penghasil energi dan zat pembentuk tubuh.",
        
        # New answers
        "Karbohidrat sangat penting karena memberikan energi, membantu pencernaan, menjaga keseimbangan tubuh, dan memberikan rasa kenyang.",  # High score
        "Fungsinya untuk tubuh supaya kuat dan sehat.",  # Low score
        "Karbohidrat membantu pencernaan dan memberi rasa kenyang karena mengandung serat.",  # Medium score
        "Membantu tubuh dalam pencernaan dan menjaga keseimbangan asam dan basa dalam tubuh.",  # High score
        "Karbohidrat itu kayak nasi dan roti, buat makan aja sih."  # Very low score
    ],
    "normalized_score": [
        # 19/100,
        # 27/100,
        # 82/100,
        # 13/100,
        # 79/100,
        91/100,  # High
        15/100,  # Low
        56/100,  # Medium
        88/100,  # High
        10/100   # Very low
    ]
}

data = {
    "reference_answer": [
        "Fungsi paru-paru adalah menukar oksigen dari udara dengan karbon dioksida dari darah, serta membantu proses pernapasan dengan mengambil oksigen dan mengeluarkan karbon dioksida."
    ] * 5,
    
    "answer": [
        "Paru-paru berfungsi untuk mengambil oksigen dan mengeluarkan karbon dioksida saat bernapas.",
        "Paru-paru membantu kita bernapas dan menyaring udara kotor.",
        "Paru-paru tempat pertukaran gas yaitu oksigen dan karbon dioksida.",
        "Fungsi paru-paru itu buat napas aja.",
        "Paru-paru menyerap oksigen dari udara dan membuat darah bersih."
    ],
    
    "normalized_score": [
        0.90,  # sangat sesuai, menyebut dua fungsi inti
        0.60,  # agak relevan tapi menyaring udara bukan fungsi utamanya
        0.80,  # sesuai, menyebut pertukaran gas
        0.40,  # terlalu umum, tidak menjelaskan mekanisme
        0.70   # cukup baik, menyebut oksigen dan darah
    ]
}

new_data = pd.DataFrame(data)
new_data.head()

,reference_answer,answer,normalized_score
0,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru berfungsi untuk mengambil oksigen da...,0.9
1,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru membantu kita bernapas dan menyaring...,0.6
2,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru tempat pertukaran gas yaitu oksigen ...,0.8
3,Fungsi paru-paru adalah menukar oksigen dari u...,Fungsi paru-paru itu buat napas aja.,0.4
4,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru menyerap oksigen dari udara dan memb...,0.7


In [47]:
transformers.utils.logging.set_verbosity_error()

model_name = "indobenchmark/indobert-lite-base-p2"
model_dir = "experiments/models/cross/bert_default/bert_default_indo_1.pt"
pipeline = inference(new_data, model_name, model_dir)
new_data[f'bert_default'] = new_data.apply(lambda x: pipeline.get_prediction(x.name)[1], axis=1)
new_data

,reference_answer,answer,normalized_score,bert_default
0,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru berfungsi untuk mengambil oksigen da...,0.9,0.49
1,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru membantu kita bernapas dan menyaring...,0.6,0.32
2,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru tempat pertukaran gas yaitu oksigen ...,0.8,0.37
3,Fungsi paru-paru adalah menukar oksigen dari u...,Fungsi paru-paru itu buat napas aja.,0.4,0.23
4,Fungsi paru-paru adalah menukar oksigen dari u...,Paru-paru menyerap oksigen dari udara dan memb...,0.7,0.38


In [48]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import numpy as np

# Ambil nilai dari dataframe
true_score = new_data["normalized_score"].values
predicted_score = new_data["bert_default"].values

# Hitung RMSE
rmse = mean_squared_error(true_score, predicted_score, squared=False)

# Hitung Pearson Correlation
correlation, _ = pearsonr(true_score, predicted_score)

# Tampilkan hasil
print("RMSE:", rmse)
print("Pearson Correlation:", correlation)


RMSE: 0.33546981980500123
Pearson Correlation: 0.9578518618342398


c:\Users\User\Documents\Code\env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
